In [ ]:
import numpy as np
import cv2

def LoG(sigma):
    n = int(np.ceil(sigma * 6))
    y, x = np.ogrid[-n//2:n//2+1, -n//2:n//2+1]
    kernel = (-1 / (np.pi * sigma**4)) * (1 - (x**2 + y**2) / (2 * sigma**2)) * np.exp(-(x**2 + y**2) / (2 * sigma**2))
    return kernel

def LoG_convolve(img):
    log_images = []
    k = 1.2 # Scaling factor, you might want to adjust this based on your needs
    for i in range(1, 10):
        sigma_1 = i * k
        filter_log = LoG(sigma_1)
        image = cv2.filter2D(img, -1, filter_log)
        image = np.pad(image, ((1, 1), (1, 1)), 'constant')
        image = np.square(image)
        log_images.append(image)
    log_image_np = np.array(log_images)
    return log_image_np

# Example usage
img = cv2.imread('/content/the_berry_farms_sunflower_field.jpeg', cv2.IMREAD_GRAYSCALE)
log_image_np = LoG_convolve(img)
print(log_image_np.shape)



(9, 1442, 1442)


In [ ]:
import numpy as np
import cv2

def detect_blob(log_image_np, sigma, k):
    co_ordinates = []
    (depth, h, w) = log_image_np.shape
    for i in range(1, h-1):
        for j in range(1, w-1):
            slice_img = log_image_np[:, i-1:i+2, j-1:j+2]
            max_value = np.amax(slice_img)
            if max_value >= 0.03:
                z, x, y = np.unravel_index(slice_img.argmax(), slice_img.shape)
                co_ordinates.append((i + x - 1, j + y - 1, k**z * sigma))
    return co_ordinates

def remove_redundant_coordinates(coordinates, threshold=0.5):
    unique_coordinates = []
    for coord in coordinates:
        is_unique = True
        for unique_coord in unique_coordinates:
            distance = np.sqrt((coord[0] - unique_coord[0])**2 + (coord[1] - unique_coord[1])**2)
            if distance < threshold * unique_coord[2]:
                is_unique = False
                break
        if is_unique:
            unique_coordinates.append(coord)
    return np.array(unique_coordinates)
sigma = 1.0
k = 1.2
coordinates = detect_blob(log_image_np, sigma, k)
unique_coordinates = remove_redundant_coordinates(coordinates, threshold=0.5)
print(unique_coordinates)


KeyboardInterrupt: ignored